How to install the package

- Open VS Code terminal in the project root:
- Run:

```
cd "c:\EC\BSE\DSDM\Term 1\21DM004 Computing for Data Science\hw5\process_data_project"
python -m pip install -e .
```

In [1]:
from process_data import (
    CSVDataLoader, DataPreprocessor,
    BMICalculator, EthnicityEncoder, GenderBinaryEncoder,
    DiabetesModel,
)
from sklearn.metrics import roc_auc_score

SyntaxError: invalid syntax (feature_transform.py, line 62)

In [ ]:
file_name = "C:/EC/BSE/DSDM/Term 1/21DM004 Computing for Data Science/hw5/process_data_project/process_data/data/sample_diabetes_mellitus_data.csv"

loader = CSVDataLoader(file_name)
train_df, test_df = loader.split_data()

pre = DataPreprocessor(train_df, test_df)
train_clean, test_clean = (
    pre.remove_nans(['age','gender','ethnicity'])
      .fill_nans_with_mean(['height','weight'])
      .get_data()
)

for t in [BMICalculator(), EthnicityEncoder(), GenderBinaryEncoder()]:
    train_clean = t.transform(train_clean)
    test_clean = t.transform(test_clean)

In [ ]:
feature_cols = ['age', 'bmi', 'gender_M', 'gender_F'] + [c for c in train_clean.columns if c.startswith('ethnicity_')]
target_col = 'diabetes_mellitus'

model = DiabetesModel(feature_columns=feature_cols, target_column=target_col, hyperparameters={'n_estimators':200,'max_depth':10,'random_state':42})
model.train(train_clean)

probs = model.predict(test_clean)
pos = probs[:,1] if probs.ndim == 2 else probs
test_clean = test_clean.copy()
test_clean['predictions_prob'] = pos
test_clean['predictions'] = (pos >= 0.5).astype(int)

roc_auc_score(test_clean[target_col], test_clean['predictions_prob'])